In [1]:
import cv2
import os
import time
from memory_queues import square_queue, gui_queue
from memory_logic import register_card
from sift_utils import extract_sift_signature, auto_crop_inside_white_edges, draw_oriented_bounding_box
from recorded_positions import pick_positions, drop_positions, home_pose, scan_pose
from pyniryo2 import NiryoRos, NiryoRobot, Vision
import pyniryo
from matplotlib import pyplot as plt
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
robot_ip_address = "169.254.200.200"
robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
# robot.activate_vacuum()
robot.tool.release_with_tool()
robot.arm.move_pose(home_pose)


In [ ]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

In [7]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

(0.26, 0.03, 0.17, 1.55, 0.2, 1.497) 


In [9]:
#L1
robot.arm.move_pose([0.26, 0.03, 0.17, 1.55, 0.2, 1.497])
#robot.tool.grasp_with_tool()

In [8]:
#HOME
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])


In [9]:
#scan pose
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585])
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.37, 0.15, 0.065, 2.50, 1.55, 2.4585])
robot.arm.move_pose([0.26, 0.03, 0.17, 1.55, 0.2, 1.497])#scan
robot.wait(5)
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])#home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585])
robot.tool.release_with_tool()
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])#home







In [25]:
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585]) #A1 pick pose
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.37, 0.15, 0.065, 2.50, 1.55, 2.4585]) #A1 drop pose
robot.arm.move_pose([0.3, 0.225, 0.06, -2.65, 1.52, -2.6712]) #L1 stack
robot.tool.release_with_tool()


In [ ]:
import time
from pyniryo2 import NiryoRobot

try:
    from recorded_positions import *
except ImportError:
    print("Error: Make sure 'recorded_positions.py' is in the same folder as your notebook.")

def collect_row_a_to_stack(robot):

    # --- 2. Define the fixed home pose ---
    home_pose = [0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]
    L1 = [0.3, 0.225, 0.06, -2.65, 1.52, -2.6712]
    # --- The list of cards to process ---
    cards_to_collect = ["A1", "A2", "A3", "A4", "A5"]

    print("--- Starting Automated Collection for Row A ---")

    # --- 3. Loop through each card from A1 to A5 ---
    for card_id in cards_to_collect:
        print(f"\n--- Processing Card: {card_id} ---")

        # Get the specific poses for the current card
        pick_pose = pick_positions[card_id]
        lift_pose = drop_positions[card_id] # Using the drop pose to lift the card up
        
        # Go to home before starting
        robot.arm.move_pose(home_pose)
        
        # Pick sequence
        print(f"Moving to pick {card_id}...")
        robot.arm.move_pose(pick_pose)
        robot.tool.grasp_with_tool()
        
        # Lift sequence
        print("Lifting card...")
        robot.arm.move_pose(lift_pose)
        robot.arm.move_pose(home_pose)
        # Go back home, then to the stack
        print("Moving to L1 stack...")
        robot.arm.move_pose([0.3, 0.225, 0.06, -2.65, 1.52, -2.6712])
        robot.tool.release_with_tool()
        robot.arm.move_pose(home_pose)

    # --- 4. Return home after the loop is finished ---
    print("\n--- Collection Complete! Returning to Home Pose. ---")
    robot.arm.move_pose(home_pose)
collect_row_a_to_stack(robot)

--- Starting Automated Collection for Row A ---

--- Processing Card: A1 ---
Moving to pick A1...
Lifting card...
Moving to L1 stack...

--- Processing Card: A2 ---


KeyboardInterrupt: 

In [12]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

(0.3, 0.22, 0.06, -2.75, 1.52, -2.6712) 


In [ ]:
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])


In [7]:
robot.tool.grasp_with_tool()

In [13]:
robot.tool.release_with_tool()


In [ ]:
robot.tool.release_with_tool()
robot.tool.grasp_with_tool()
robot.arm.move_pose([-0.02, -0.46, 0.18, -0.13, -1.15, -1.504])


In [ ]:
import cv2
import pyniryo
from pyniryo2 import NiryoRos, Vision
import numpy as np

CARD_BOX = (270, 140, 190, 190)  # x,y,h,w

ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

print("[SCAN] Looking for card at ")
last_center = stable_since = detection_time = None
while True:
    img_compressed = vision.get_img_compressed()
    
    # Check if the image was successfully captured
    if img_compressed is None:
        print("[ERROR] Could not get compressed image from the robot.")
        continue  # Skip the rest of the loop and try again

    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    
    # Add a check for the uncompressed image as well
    if img_uncompressed is None:
        print("[ERROR] Failed to uncompress image.")
        continue

    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    frame = cv2.resize(img, (640, 480))
    cv2.rectangle(frame, (CARD_BOX[0], CARD_BOX[1]), (CARD_BOX[0] + CARD_BOX[2], CARD_BOX[1] + CARD_BOX[3]), (0, 255, 0), 2)
    cv2.imshow("Adjust ROI", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

[SCAN] Looking for card at 


KeyboardInterrupt: 

: 

In [ ]:
import cv2
from pyniryo2 import NiryoRobot
import pyniryo  # Import the pyniryo helper library
import numpy as np

# --- Configuration ---
ROBOT_IP_ADDRESS = "169.254.200.200"
# Defines the region of interest (ROI) in the camera feed (x, y, w, h)
CARD_BOX = (270, 140, 190, 190)

def show_camera_feed():
    """
    Connects to the Niryo robot and displays its camera feed in a window.
    Draws a green rectangle to show the Region of Interest (ROI).
    """
    robot = None  # Initialize robot to None
    try:
        # Connect to the robot
        robot = NiryoRobot(ROBOT_IP_ADDRESS)
        print("✅ Successfully connected to the robot.")
    except Exception as e:
        print(f"❌ Could not connect to robot at {ROBOT_IP_ADDRESS}. Error: {e}")
        return

    print("\nDisplaying camera feed. Press 'q' in the window to quit.")

    try:
        while True:
            # --- SIMPLIFIED IMAGE CAPTURE LOGIC ---
            # 1. Get the compressed image from the robot's camera
            img_compressed = robot.vision.get_img_compressed()
            if img_compressed is None:
                print("[ERROR] Could not get compressed image from the robot.")
                cv2.waitKey(500)
                continue

            # 2. Uncompress the image using the pyniryo helper
            # This will be our main image for this simplified test.
            img_np = pyniryo.uncompress_image(img_compressed)
            if img_np is None:
                print("[ERROR] Failed to uncompress image.")
                cv2.waitKey(500)
                continue

            # NOTE: We are skipping the undistortion step to simplify the process
            # and isolate potential issues. The image may appear slightly warped.
            # --- END OF SIMPLIFICATION ---

            # Resize the image for display
            frame = cv2.resize(img_np, (640, 480))

            # Draw the green rectangle for the Region of Interest
            pt1 = (CARD_BOX[0], CARD_BOX[1])
            pt2 = (CARD_BOX[0] + CARD_BOX[2], CARD_BOX[1] + CARD_BOX[3])
            cv2.rectangle(frame, pt1, pt2, (0, 255, 0), 2)

            # Show the image in a window
            cv2.imshow("Niryo Camera Feed - Adjust ROI (Press 'q' to quit)", frame)

            # Wait for a key press. If 'q' is pressed, exit the loop.
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\n'q' pressed. Closing camera feed.")
                break

    except Exception as e:
        print(f"An error occurred in the main loop: {e}")
    finally:
        # Clean up and close all OpenCV windows and disconnect from the robot
        cv2.destroyAllWindows()
        if robot:
            robot.close_connection()
        print("Camera feed closed and disconnected from the robot.")


if __name__ == '__main__':
    show_camera_feed()

